In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR # Common options

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


100%|██████████| 170M/170M [00:19<00:00, 8.93MB/s] 


In [3]:
# A simple block that can be Residual or Plain
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, use_residual=True):
        super(Block, self).__init__()
        self.use_residual = use_residual
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # If dimensions change, we need to match them for the skip connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        # KEY PART: The Architectural Stress Test
        if self.use_residual:
            out += self.shortcut(x)  # This is the "ResNet"
            
        out = F.relu(out)
        return out

# The Full Network
class StressTestNet(nn.Module):
    def __init__(self, use_residual=True):
        super(StressTestNet, self).__init__()
        self.in_channels = 16
        
        # Initial convolution
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        
        # Create a deep network (3 layers per block type)
        self.layer1 = self._make_layer(16, 3, stride=1, use_residual=use_residual)
        self.layer2 = self._make_layer(32, 3, stride=2, use_residual=use_residual)
        self.layer3 = self._make_layer(64, 3, stride=2, use_residual=use_residual)
        
        self.fc = nn.Linear(64, 10)

    def _make_layer(self, out_channels, blocks, stride, use_residual):
        layers = []
        layers.append(Block(self.in_channels, out_channels, stride, use_residual))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(Block(out_channels, out_channels, stride=1, use_residual=use_residual))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [5]:
def get_gradient_norms(model):
    total_norm = 0.0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm ** 0.5
    return total_norm

In [4]:
import torch

if torch.cuda.is_available():
    print(f"✅ Success! GPU Detected: {torch.cuda.get_device_name(0)}")
    print(f"Memory Usage: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
else:
    print("❌ GPU not found. You are running on CPU.")

✅ Success! GPU Detected: Tesla T4
Memory Usage: 0.00 GB


In [ ]:
# Define your hyperparameters for the heatmap
learning_rates = [0.01, 0.005, 0.001]
batch_sizes = [32, 64, 128]  # Note: 4 is usually too unstable for SGD stress testing
architectures = [True, False] # True = ResNet, False = Plain Net (Stress Test)
schedulers = ["None", "StepLR", "Cosine"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

# Store results: (use_res, batch_size, lr) -> final_accuracy
results = {}

for use_res in architectures:
    model_type = "ResNet" if use_res else "PlainNet"
    
    for batch_size in batch_sizes:
        # Re-initialize DataLoaders for each batch size
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
        
        for lr in learning_rates:

            for sched in schedulers:
            
                # 1. Initialize Model
                model = StressTestNet(use_residual=use_res).to(device)
                
                # 2. Initialize SGD (NO ADAM allowed here!)
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
                criterion = nn.CrossEntropyLoss()

                if (sched == "StepLR"):
                    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
                elif (sched == "Cosine"):
                    scheduler = CosineAnnealingLR(optimizer, T_max=100)
                # 3. Short Training Loop (e.g., 5 epochs for the heatmaps)
                model.train()
                run_name = f"runs/{model_type}_BS{batch_size}_LR{lr}_SCHDL{sched}_SGD"
                writer = SummaryWriter(run_name)
                print(f"Starting Run: {run_name}")

                step = 0
                for epoch in range(5): 
                    running_loss = 0.0
                    for i, (inputs, labels) in enumerate(trainloader):
                        inputs, labels = inputs.to(device), labels.to(device)
                        
                        optimizer.zero_grad()
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        
                        # Track gradients before stepping
                        grad_norm = get_gradient_norms(model)
                        writer.add_scalar('StressTest/GradientNorm', grad_norm, step)
                        writer.add_scalar('Training/Loss', loss.item(), step)
                        
                        optimizer.step()
                    if sched != "None":
                        scheduler.step()
                
                writer.add_hparams(
                    {'lr': lr, 'bsize': batch_size, 'residual': use_res},
                    {'hparam/loss': loss.item(), 'hparam/grad_norm': grad_norm}
                )
                
                writer.close()
                print(f"--> Done. Final Loss: {loss.item():.4f}")

print("Experiment Complete. Run 'tensorboard --logdir=runs' to view.")

Running on: cuda
Starting Run: runs/ResNet_BS32_LR0.01_SCHDLStepLR_SGD
--> Done. Final Loss: 0.4855
Starting Run: runs/ResNet_BS32_LR0.01_SCHDLCosine_SGD
--> Done. Final Loss: 0.2021
Starting Run: runs/ResNet_BS32_LR0.005_SCHDLStepLR_SGD
--> Done. Final Loss: 0.8363
Starting Run: runs/ResNet_BS32_LR0.005_SCHDLCosine_SGD
--> Done. Final Loss: 0.1758
Starting Run: runs/ResNet_BS32_LR0.001_SCHDLStepLR_SGD
--> Done. Final Loss: 0.3675
Starting Run: runs/ResNet_BS32_LR0.001_SCHDLCosine_SGD
--> Done. Final Loss: 0.6381
Starting Run: runs/ResNet_BS64_LR0.01_SCHDLStepLR_SGD
--> Done. Final Loss: 0.2822
Starting Run: runs/ResNet_BS64_LR0.01_SCHDLCosine_SGD
--> Done. Final Loss: 0.7775
Starting Run: runs/ResNet_BS64_LR0.005_SCHDLStepLR_SGD
--> Done. Final Loss: 0.5817
Starting Run: runs/ResNet_BS64_LR0.005_SCHDLCosine_SGD


KeyboardInterrupt: 